In [1]:
import numpy as np
import optuna
import joblib
import ast
from sklearn.metrics import f1_score
import os

In [2]:

def criterion_evaluation(filename, max_round, criterion, thresholds):
    f1_scores = []
    for fold in range(5):
        with open(f"{filename}_{fold}.txt", "r") as f:
            true_labels = []
            classifications = []
            for line in f:
                splt = line[:-1].split(" ")
                user, label = ast.literal_eval(splt[0].replace("(","('").replace(",","',"))
                true_labels.append(label)
                splt = [ast.literal_eval(t) for t in splt[1:90+1]]
                classification = 0
                count = 0
                for i, (pred, conf) in enumerate(splt):
                    if (i + 10)>max_round:
                        break
                    if criterion == "confidence":#  and i%10==0:
                        if pred==1 and conf >= thresholds[0] :
                            classification= 1
                    if criterion == "ratio" and conf >=thresholds[0]:# and i%10==0:
                        count = count + pred
                        if count/(10 + i) >= thresholds[1]:
                            classification=1
                    if criterion == "confidence+" and conf >=thresholds[0]:# and i%10==0:
                        count = count + pred
                        if count >= thresholds[1]:
                            classification=1
                    if criterion == "consecutive":# and i%10==0: ##added stride
                        if pred==1 and conf >=thresholds[0]:
                            count = count + 1
                        else:
                            count = 0
                        if count == thresholds[1]:
                            classification = 1
                    
                classifications.append(classification)
        f1_scores.append(f1_score(true_labels,classifications))
    return f1_scores

In [3]:
max_round=100


In [4]:
for FEATURE_TYPE in ['BoW','WE','LM','NN']:
    print(FEATURE_TYPE)
    main_threshold=0
    def objective_confidence(trial):
        params = {
            0: trial.suggest_float("confidence_threshold", 0.5, 1)
        }
        return np.mean(criterion_evaluation(FEATURE_TYPE,max_round,"confidence", params))
        
    if not os.path.exists(f"Thresholds2/{FEATURE_TYPE}_confidence_{max_round}.pkl"):
        study = optuna.create_study(study_name=f"{FEATURE_TYPE}_confidence",direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.NopPruner())
        study.optimize(objective_confidence, n_trials=50, timeout=(60*60*3), gc_after_trial=True)
        joblib.dump(study, f"Thresholds2/{FEATURE_TYPE}_confidence_{max_round}.pkl")
    study = joblib.load(f"Thresholds2/{FEATURE_TYPE}_confidence_{max_round}.pkl")
    print(f"confidence threshold:{study.best_trial.params}    f1:{study.best_value}") 

    def objective_ratio(trial):
        params = {
            0: trial.suggest_float("confidence_threshold", 0.5, 1),
            1: trial.suggest_float("ratio_threshold", 0.01, 1)
        }
        return np.mean(criterion_evaluation(FEATURE_TYPE,max_round,"ratio", params))
    
    if not os.path.exists(f"Thresholds2/{FEATURE_TYPE}_ratio_{max_round}.pkl"):
        
        study2 = optuna.create_study(study_name=f"{FEATURE_TYPE}_ratio",direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.NopPruner())
        study2.optimize(objective_ratio, n_trials=50, timeout=(60*60*3), gc_after_trial=True)
        joblib.dump(study2, f"Thresholds2/{FEATURE_TYPE}_ratio_{max_round}.pkl")
    study2 = joblib.load(f"Thresholds2/{FEATURE_TYPE}_ratio_{max_round}.pkl")
    print(f"ratio threshold:{study2.best_trial.params}    f1:{study2.best_value}") 

    def objective_consecutive(trial):
        params = {
            0: trial.suggest_float("confidence_threshold", 0.5, 1),
            1: trial.suggest_int("consecutive_threshold", 2, 100)
        }
        return np.mean(criterion_evaluation(FEATURE_TYPE,max_round,"consecutive", params))
    if not os.path.exists(f"Thresholds2/{FEATURE_TYPE}_consecutive_{max_round}.pkl"):

        study3 = optuna.create_study(study_name=f"{FEATURE_TYPE}_consecutive",direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.NopPruner())
        study3.optimize(objective_consecutive, n_trials=50, timeout=(60*60*3), gc_after_trial=True)
        joblib.dump(study3, f"Thresholds2/{FEATURE_TYPE}_consecutive_{max_round}.pkl")
    study3 = joblib.load(f"Thresholds2/{FEATURE_TYPE}_consecutive_{max_round}.pkl")
    
    print(f"consecutive threshold:{study3.best_trial.params}    f1:{study3.best_value}") 

    def objective_confidenceplus(trial):
        params = {
            0: trial.suggest_float("confidence_threshold", 0.5, 1),
            1: trial.suggest_int("amount_threshold", 2, 100)
        }
        return np.mean(criterion_evaluation(FEATURE_TYPE,max_round,"confidence+", params))
    if not os.path.exists(f"Thresholds2/{FEATURE_TYPE}_confidence+_{max_round}.pkl"):

        study4 = optuna.create_study(study_name=f"{FEATURE_TYPE}_confidence+",direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.NopPruner())
        study4.optimize(objective_confidenceplus, n_trials=50, timeout=(60*60*3), gc_after_trial=True)
        joblib.dump(study4, f"Thresholds2/{FEATURE_TYPE}_confidence+_{max_round}.pkl")
    study4 = joblib.load(f"Thresholds2/{FEATURE_TYPE}_confidence+_{max_round}.pkl")
    
    print(f"confidence+ threshold:{study4.best_trial.params}    f1:{study4.best_value}") 


BoW
confidence threshold:{'confidence_threshold': 0.981097006799505}    f1:0.747765104945598
ratio threshold:{'confidence_threshold': 0.9418006589364453, 'ratio_threshold': 0.2164885680519905}    f1:0.7257421069677739
consecutive threshold:{'confidence_threshold': 0.9511499616821109, 'consecutive_threshold': 7}    f1:0.7150801222055534
confidence+ threshold:{'confidence_threshold': 0.9670048907766651, 'amount_threshold': 2}    f1:0.736136831657834
WE
confidence threshold:{'confidence_threshold': 0.9915594348630237}    f1:0.31167441456173584
ratio threshold:{'confidence_threshold': 0.9010355977206922, 'ratio_threshold': 0.5009682728717357}    f1:0.5797055077701029
consecutive threshold:{'confidence_threshold': 0.9154925752572305, 'consecutive_threshold': 19}    f1:0.5214594489765277
confidence+ threshold:{'confidence_threshold': 0.9467049364831088, 'amount_threshold': 14}    f1:0.5358996984828187
LM
confidence threshold:{'confidence_threshold': 0.9806946886180349}    f1:0.83334686937842